<a href="https://colab.research.google.com/github/yashlal/Deepfake-Microbiomes/blob/main/Combination.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from newsolver import predict_community_fullnp
import numpy as np
import pandas as pd
import random as rd
from numba import njit
from numba.typed import List
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import ReduceLROnPlateau
import pickle
import torch.optim as optim
import time
from math import sqrt
import matplotlib.pyplot as plt

data = pd.read_excel('RealData.xlsx', index_col=0)
specs = data.columns.tolist()
trimmed_specs = []

for i in range(len(specs)):
    if data.iloc[:,i].astype(bool).sum() >= 85:
        trimmed_specs.append(specs[i])
dim1 = len(trimmed_specs)

typed_trimmed_specs = List()
[typed_trimmed_specs.append(x) for x in trimmed_specs]

@njit()
def get_LT(full_ar):
    ar = []
    for i in range(len(full_ar)):
        for j in range(i):
            ar.append(full_ar[i][j])
    return ar

@njit()
def generate_matrix(comm, tolerance):
    dim = len(comm)
    ar = np.zeros((dim,dim))

    for i in range(dim):
        for j in range(i+1):
            if i == j:
                ar[i][j] = 0
            else:
                r = rd.random()
                # m = mult[i*dim1+j]
                ar[i][j] = r
                ar[j][i] = (1-r)

    return ar

def datagen():
    lm = generate_matrix(typed_trimmed_specs, 0)
    cm = predict_community_fullnp(lm, trimmed_specs, verb=False)
    return (cm, get_LT(lm))

# select CUDA if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if str(device) == 'cuda:0':
	print('CUDA device selected!')
elif str(device) == 'cpu':
	print('CUDA device not available. CPU selected')

class MyNet(nn.Module):
    def __init__(self):
        super(MyNet, self).__init__()
        self.fc1 = nn.Linear(462, 462*10)
        self.fc2 = nn.Linear(462*10, 231*461)
    def forward(self,x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

net = MyNet().to(device)
criterion = nn.MSELoss(reduction='mean')
optimizer = optim.Adam(net.parameters(), lr=1e-4)
scheduler = ReduceLROnPlateau(optimizer, patience=12, threshold=1e-3, verbose=True)

mytest_x = []
mytest_y = []

for i in range(25):
      x, y = datagen()
      mytest_x.append(torch.from_numpy(x).float().to(device))
      mytest_y.append(torch.FloatTensor(y).to(device))

def test_net(test_x, test_y):
    test_loss = 0
    for i in range(len(test_x)):
      input, true_y = test_x[0], test_y[1]

      output = net(input).to(device)
      loss = criterion(output, true_y).to(device)
      test_loss += sqrt(loss.item())

    print(f'Average Test RMS: {test_loss/len(test_x)}')

loss_v = []

for i in range(20000):
    if (i%1000==0) and (i!=0):
      test_net(mytest_x, mytest_y)
      continue

    optimizer.zero_grad()
    x, y = datagen()
    input = torch.from_numpy(x).float().to(device)
    true_y = torch.FloatTensor(y).to(device)

    output = net(input).to(device)

    loss = criterion(output, true_y).to(device)
    s = sqrt(loss.item())
    print(f'Epoch {i}: {s}')
    loss_v.append(s)
    loss.backward()

    optimizer.step()
    scheduler.step(s)

    


PATH = 'model.pth'
torch.save(net.state_dict(), PATH)

plt.plot(loss_v[70:])
plt.savefig('Loss.png')
plt.show()


CUDA device selected!
Epoch 0: 0.5768879982175149
Epoch 1: 0.5716913876966935
Epoch 2: 0.5694755526964632
Epoch 3: 0.5653926857023491
Epoch 4: 0.5607352123138913
Epoch 5: 0.5585463443727652
Epoch 6: 0.5544540021270483
Epoch 7: 0.5499496404393992
Epoch 8: 0.5446007989282874
Epoch 9: 0.5411297784454646
Epoch 10: 0.5354922277169688
Epoch 11: 0.5328085141298544
Epoch 12: 0.5285590966638231
Epoch 13: 0.5249827904378294
Epoch 14: 0.5213469420652415
Epoch 15: 0.5155810424100641
Epoch 16: 0.5124035977202293
Epoch 17: 0.5075634859336443
Epoch 18: 0.5013282835087608
Epoch 19: 0.49936856691802595
Epoch 20: 0.49300084198537425
Epoch 21: 0.49005638934231016
Epoch 22: 0.48384377359233627
Epoch 23: 0.48064586817327476
Epoch 24: 0.47483164288273577
Epoch 25: 0.47174428809406954
Epoch 26: 0.46660924676034266
Epoch 27: 0.46201789447057123
Epoch 28: 0.45754374599803566
Epoch 29: 0.45396861109216446
Epoch 30: 0.44939851895624333
Epoch 31: 0.44396662193137915
Epoch 32: 0.43886579471735526
Epoch 33: 0.43506